<a href="https://colab.research.google.com/github/YOUGALAKSHMI/Machine_Learning/blob/main/music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving user_text.csv to user_text.csv


In [ ]:
# Use this after uploading the file
df = pd.read_csv('user_text.csv', encoding='ISO-8859-1')

In [ ]:
df.head()

,text,mood
0,The sun is shining and I feel great.,happy
1,I just got a new puppy and I'm thrilled!,happy
2,What a beautiful day to go outside.,happy
3,"Everything is fine, just the usual stuff.",bored
4,"Everything is fine, just the usual stuff.",bored


In [ ]:
df.rename(columns={'text_column_name': 'text', 'mood_column_name': 'mood'}, inplace=True)

In [ ]:
import nltk
import string
from nltk.corpus import stopwords

In [ ]:
# Download stopwords list
nltk.download('stopwords')

# Set of stopwords and punctuation
stop_words = set(stopwords.words('english'))
punctuations = string.punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = ''.join(char for char in text if char not in punctuations)
    # Remove stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

In [ ]:
# Apply preprocessing to the text column
df['clean_text'] = df['text'].apply(preprocess_text)

In [ ]:
# Check the result
df[['text', 'clean_text']].head()

,text,clean_text
0,The sun is shining and I feel great.,sun shining feel great
1,I just got a new puppy and I'm thrilled!,got new puppy im thrilled
2,What a beautiful day to go outside.,beautiful day go outside
3,"Everything is fine, just the usual stuff.",everything fine usual stuff
4,"Everything is fine, just the usual stuff.",everything fine usual stuff


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        2240 non-null   object
 1   mood        2240 non-null   object
 2   clean_text  2240 non-null   object
dtypes: object(3)
memory usage: 52.6+ KB


In [ ]:
df.describe()

,text,mood,clean_text
count,2240,2240,2240
unique,203,6,195
top,Why does nothing ever go my way?,neutral,nothing ever go way
freq,122,614,122


In [ ]:
df['mood'].value_counts()

,count
mood,
neutral,614
happy,589
bored,439
angry,313
sad,215
excited,70


In [ ]:
from sklearn.utils import resample

# Separate minority class
excited_df = df[df['mood'] == 'excited']

# Separate majority classes
other_df = df[df['mood'] != 'excited']

# Upsample "excited" to match around 400 samples
excited_upsampled = resample(
    excited_df,
    replace=True,     # sample with replacement
    n_samples=400,     # desired number of samples
    random_state=42
)

In [ ]:
# Combine datasets
df_balanced = pd.concat([other_df, excited_upsampled])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check new distribution
print(df_balanced['mood'].value_counts())


mood
neutral    614
happy      589
bored      439
excited    400
angry      313
sad        215
Name: count, dtype: int64


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# TF-IDF vectorization
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df_balanced['clean_text'])
y = df_balanced['mood']

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Train Logistic Regression
model = LogisticRegression(max_iter=300)
model.fit(X_train, y_train)

LogisticRegression(max_iter=300)

In [ ]:
# Predict and evaluate
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9727626459143969
Classification Report:
               precision    recall  f1-score   support

       angry       0.98      0.98      0.98        62
       bored       1.00      0.94      0.97        88
     excited       0.86      1.00      0.92        80
       happy       1.00      0.98      0.99       118
     neutral       1.00      0.98      0.99       123
         sad       1.00      0.91      0.95        43

    accuracy                           0.97       514
   macro avg       0.97      0.97      0.97       514
weighted avg       0.98      0.97      0.97       514



In [ ]:
mood_to_music = {
    'happy': {
        'positive': [
            ('Happy - Pharrell Williams', 'https://open.spotify.com/track/60nZcImufyMA1MKQY3dcCH'),
            ('Good Life - OneRepublic', 'https://open.spotify.com/track/4kLLWzv94UqvRDigfDgG7d')
        ],
    },
    'sad': {
        'positive': [
            ('Shake It Off - Taylor Swift', 'https://open.spotify.com/track/5WfhXhM7vbx5kOdUWv7608'),
            ('Stronger - Kelly Clarkson', 'https://open.spotify.com/track/1R0a2iXumgCiFb7HEZ7gUE')
        ],
        'negative': [
            ('Sad Melodies', 'https://open.spotify.com/playlist/37i9dQZF1DX3bcfiyW6qms'),
            ('Fix You - Coldplay', 'https://open.spotify.com/track/7LVHVU3tWfcxj5aiPFEW4Q')
        ]
    },
    'angry': {
        'positive': [
            ('Beautiful Day - U2', 'https://open.spotify.com/track/0wJoRSUAOuLclbUQMyZq7I'),
            ('Can’t Stop The Feeling - Justin Timberlake', 'https://open.spotify.com/track/6JV2JOEocMgcZxYSZelKcc')
        ],
        'negative': [
            ('In The End - Linkin Park', 'https://open.spotify.com/track/60a0Rd6pjrkxjPbaKzXjfq'),
            ('Numb - Linkin Park', 'https://open.spotify.com/track/2nLtzopw4rPReszdYBJU6h')
        ]
    },
    'bored': {
        'positive': [
            ('Uptown Funk - Bruno Mars', 'https://open.spotify.com/track/32OlwWuMpZ6b0aN2RZOeMS'),
            ('Dance Monkey - Tones and I', 'https://open.spotify.com/track/2XU0oxnq2qxCpomAAuJY8K')
        ],
        'negative': [
            ('The Lazy Song - Bruno Mars', 'https://open.spotify.com/track/0Hzj3qOem4eN0gh1aF9FzJ'),
            ('Bored - Billie Eilish', 'https://open.spotify.com/track/3jjujdWJ72nww5eGnfs2E7')
        ]
    },
    'neutral': {
        'positive': [
            ('Counting Stars - OneRepublic', 'https://open.spotify.com/track/2EqlS6tkEnglzr7tkKAAYD'),
            ('Budapest - George Ezra', 'https://open.spotify.com/track/5jrdCoLpJSvHHorevXBATy')
        ],
        'negative': [
            ('Lost Boy - Ruth B.', 'https://open.spotify.com/track/3U4isOIWM3VvDubwSI3y7a'),
            ('I’m Not The Only One - Sam Smith', 'https://open.spotify.com/track/2nGFzvICaeEWjIrBrL2RAx')
        ]
    },
    'excited': {
        'positive': [
            ('Sad Melodies', 'https://open.spotify.com/playlist/37i9dQZF1DX3bcfiyW6qms'),
            ('Don’t Start Now - Dua Lipa', 'https://open.spotify.com/track/6WrI0LAC5M1Rw2MnX2ZvEg')
        ],

    }
}

In [ ]:
def recommend_music_conversation(user_input, user_reply):
    clean_input = preprocess_text(user_input)
    vectorized_input = tfidf.transform([clean_input])
    predicted_mood = model.predict(vectorized_input)[0]

    print(f"\n🤖 I think you're feeling **{predicted_mood}**.")
    print("Would you like a song suggestion?")

    if user_reply.lower() in ['yes', 'y']:
        mood_data = mood_to_music.get(predicted_mood, {})

        if predicted_mood in ['angry', 'sad', 'bored']:
            print("\nDo you want to stay in your current mood or uplift it?")
            print("1. Stay in mood")
            print("2. Uplift mood")
            choice = input("Enter 1 or 2: ")

            songs = mood_data.get('negative' if choice == '1' else 'positive', [])
        else:
            songs = mood_data.get('positive', [])

        print("\n🎧 Here's your playlist suggestion:")
        for title, link in songs:
            print(f"🎵 {title}: {link}")
    else:
        print("No worries! Come back for a vibe-check anytime 🎶")

In [ ]:
user_text = input("How are you feeling today? Describe your mood: ")
user_wants_music = input("Would you like a music recommendation? (yes/no): ")

recommend_music_conversation(user_text, user_wants_music)

How are you feeling today? Describe your mood: cryy
Would you like a music recommendation? (yes/no): y

🤖 I think you're feeling **excited**.
Would you like a song suggestion?

🎧 Here's your playlist suggestion:
🎵 Sad Melodies: https://open.spotify.com/playlist/37i9dQZF1DX3bcfiyW6qms
🎵 Don’t Start Now - Dua Lipa: https://open.spotify.com/track/6WrI0LAC5M1Rw2MnX2ZvEg
